In [1]:
import pandas as pd
import os
import wiggum as wg
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [2]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the LabeledDataFrame with that DataFrame to create the object

In [3]:
labeled_df_setup = wg.LabeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [4]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [5]:
roles = {'state':['trend','groupby'], 'year':'trend', 'driver_gender':['trend','groupby'], 
         'driver_race':['trend','groupby'],
       'decriminalization':['groupby'], 'medical':['groupby'],
         'recreational':['groupby'], 'no_reforms':['groupby'],
       'search_conducted_false':'ignore', 'search_conducted_true':'ignore',
       'search_conducted_rate':'trend', 'contraband_found_false':'ignore',
       'contraband_found_true':'ignore', 'contraband_found_rate':'trend', 'hit_false':'ignore',
       'hit_true':'ignore', 'hit_rate':'trend', 'num_stops':'trend'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
var_types = {'driver_gender':'categorical','decriminalization':'categorical', 
             'medical':'categorical', 'recreational':'categorical', 'no_reforms':False,}
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops',
             'contraband_found_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [6]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.set_weighting_vars(weighting)
labeled_df_setup.set_var_types(var_types)
labeled_df_setup.meta_df

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:285: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.meta_df['role'][k] = v
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:261: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.meta_df['var_type'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,trend,False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,ignore,True,NaN


Now, we've set this up, we can also save these configurations to load them in directly in the future

In [7]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

True

We can see what this does, using a bash magic

In [8]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [9]:
labeled_df = wg.LabeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [10]:
labeled_df.add_intersectional(['driver_gender','driver_race'])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0,F_White
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0,M_White
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian


We can also pass more variables and lengths of the tuples that we want it to combine.  For example if we provide 3 categorical variables we can do both pairs and triples of the variables.

In [11]:
labeled_df.add_intersectional(['driver_gender','driver_race','state'],[2,3])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race,driver_gender_state,driver_race_state,driver_gender_driver_race_state
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,NaN,NaN,1.0,NaN,NaN,31.0,F_White,F_AZ,White_AZ,F_White_AZ
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black,M_AZ,Black_AZ,M_Black_AZ
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic,M_AZ,Hispanic_AZ,M_Hispanic_AZ
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,0.055556,2.0,2.0,0.5,36.0,M_White,M_AZ,White_AZ,M_White_AZ
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian,F_AZ,Asian_AZ,F_Asian_AZ


In [12]:
labeled_df.add_all_dpgmm(qual_thresh =.2)
labeled_df.df.head()

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race,driver_gender_state,driver_race_state,driver_gender_driver_race_state,search_conducted_rate_num_stops_dpgmm
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,NaN,1.0,NaN,NaN,31.0,F_White,F_AZ,White_AZ,F_White_AZ,1
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,0.333333,1.0,1.0,0.5,3.0,M_Black,M_AZ,Black_AZ,M_Black_AZ,5
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,NaN,1.0,NaN,NaN,15.0,M_Hispanic,M_AZ,Hispanic_AZ,M_Hispanic_AZ,1
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,0.055556,2.0,2.0,0.5,36.0,M_White,M_AZ,White_AZ,M_White_AZ,5
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,NaN,3.0,NaN,NaN,61.0,F_Asian,F_AZ,Asian_AZ,F_Asian_AZ,1


In [13]:
labeled_df.add_quantile(['hit_rate','num_stops'])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,hit_true,hit_rate,num_stops,driver_gender_driver_race,driver_gender_state,driver_race_state,driver_gender_driver_race_state,search_conducted_rate_num_stops_dpgmm,hit_ratequantiles,num_stopsquantiles
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,NaN,NaN,31.0,F_White,F_AZ,White_AZ,F_White_AZ,1,high,low
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,0.5,3.0,M_Black,M_AZ,Black_AZ,M_Black_AZ,5,high,low
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,NaN,NaN,15.0,M_Hispanic,M_AZ,Hispanic_AZ,M_Hispanic_AZ,1,high,low
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,0.5,36.0,M_White,M_AZ,White_AZ,M_White_AZ,5,high,low
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,NaN,NaN,61.0,F_Asian,F_AZ,Asian_AZ,F_Asian_AZ,1,high,low


In [14]:
labeled_df.get_vars_per_type('categorical')

['state',
 'driver_gender',
 'driver_race',
 'decriminalization',
 'medical',
 'recreational',
 'driver_gender_driver_race',
 'driver_gender_state',
 'driver_race_state',
 'driver_gender_driver_race_state',
 'search_conducted_rate_num_stops_dpgmm',
 'hit_ratequantiles',
 'num_stopsquantiles']

In [15]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [16]:
corrobj = wg.All_Pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

['year',
 'search_conducted_rate',
 'contraband_found_rate',
 'hit_rate',
 'num_stops']

In [17]:
rankobj = wg.Mean_Rank_Trend()
linreg_obj = wg.All_Linear_Trend()

# Computing Trends on a LabeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [18]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()

,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_strength,trend_type,agg_trend,agg_trend_strength,comparison_type
0,year,search_conducted_rate,state,AZ,-0.247018,0.247018,pearson_corr,-0.03903,0.03903,aggregate-subgroup
1,year,search_conducted_rate,state,CO,-0.414566,0.414566,pearson_corr,-0.03903,0.03903,aggregate-subgroup
2,year,search_conducted_rate,state,CT,0.118238,0.118238,pearson_corr,-0.03903,0.03903,aggregate-subgroup
3,year,search_conducted_rate,state,IL,-0.199765,0.199765,pearson_corr,-0.03903,0.03903,aggregate-subgroup
4,year,search_conducted_rate,state,MA,-0.603026,0.603026,pearson_corr,-0.03903,0.03903,aggregate-subgroup


Now we can use a list of objects and apply multiple trends

In [19]:
labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/trend_components/regression.py:119: UserWarning: cannot compute with two different weights
  warnings.warn('cannot compute with two different weights')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:249: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:249: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/d

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup_trend,subgroup_trend_strength,trend_type
2995,"[RI, VT, WI, CT, MD, MA, AZ, CO, IL, NC, WA, S...",0.320900,aggregate-subgroup,num_stops,state,driver_race_state,White_WI,[WI],NaN,rank_trend
1122,"[F, M]",0.043300,aggregate-subgroup,hit_rate,driver_gender,driver_race_state,Asian_CT,"[F, M]",0.219200,rank_trend
2330,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, WA, RI, I...",0.388100,aggregate-subgroup,search_conducted_rate,state,driver_gender_driver_race_state,M_Hispanic_VT,[VT],NaN,rank_trend
681,"[F, M]",0.179700,aggregate-subgroup,contraband_found_rate,driver_gender,driver_gender_driver_race_state,F_Hispanic_CO,[F],NaN,rank_trend
1195,0.807915,0.807915,aggregate-subgroup,search_conducted_rate,contraband_found_rate,driver_race_state,White_SC,0.834941,0.834941,pearson_corr
1041,"[Asian, White, Other, Hispanic, Black]",0.071600,aggregate-subgroup,contraband_found_rate,driver_race,driver_gender_driver_race_state,M_White_SC,[White],NaN,rank_trend
2666,-0.0421016,0.042102,aggregate-subgroup,hit_rate,num_stops,driver_gender_driver_race_state,M_White_WA,0.415494,0.415494,pearson_corr
380,"[Asian, White, Other, Black, Hispanic]",0.277200,aggregate-subgroup,search_conducted_rate,driver_race,driver_race_state,White_MA,[White],NaN,rank_trend
969,"[Asian, White, Other, Hispanic, Black]",0.071600,aggregate-subgroup,contraband_found_rate,driver_race,driver_gender_driver_race_state,F_White_AZ,[White],NaN,rank_trend
1854,-0.0655937,0.065594,aggregate-subgroup,search_conducted_rate,num_stops,driver_gender_driver_race_state,M_White_IL,0.567533,0.567533,pearson_corr


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

We can see what types of trends were computed from `result_df`

In [20]:
pd.unique(labeled_df.result_df['trend_type'])

array(['pearson_corr', 'rank_trend', 'lin_reg'], dtype=object)

In [21]:
pd.unique(labeled_df.result_df['comparison_type'])

array(['aggregate-subgroup'], dtype=object)

We can also add trends that are structured for pairwise comparisons

In [22]:
labeled_df.get_pairwise_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:342: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:342: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:342: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:342: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:342: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:342: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:342: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:342: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/

Index(['feat1', 'feat2', 'subgroup_trend', 'subgroup_trend_strength',
       'group_feat', 'subgroup', 'trend_type'],
      dtype='object')
Index(['feat1', 'feat2', 'group_feat', 'subgroup', 'subgroup_trend',
       'subgroup_trend_strength', 'trend_type'],
      dtype='object')
14312
2106184
Index(['feat1', 'feat2', 'group_feat', 'subgroup', 'subgroup_trend',
       'subgroup_trend_strength', 'trend_type', 'subgroup2', 'subgroup_trend2',
       'subgroup_trend_strength2'],
      dtype='object')
1040120
813050


,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type
603854,NaN,NaN,pairwise,search_conducted_rate,driver_race,driver_gender_driver_race_state,M_Asian_AZ,M_Asian_NC,[Asian],[Asian],NaN,NaN,rank_trend
1457160,NaN,NaN,pairwise,year,num_stops,driver_gender_driver_race_state,F_Hispanic_SC,F_Other_VT,194.192,5,0.568865,1.000000,lin_reg
1862985,NaN,NaN,pairwise,num_stops,hit_rate,driver_gender_driver_race_state,M_Black_NC,M_Other_WA,0.184235,0.351846,453.549997,39.076437,lin_reg
978309,NaN,NaN,pairwise,hit_rate,driver_gender,driver_gender_driver_race_state,M_White_MA,M_White_VT,[M],[M],NaN,NaN,rank_trend
3155,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,White,NaN,-5591.73,NaN,0.097383,NaN,lin_reg
1178075,NaN,NaN,pairwise,num_stops,driver_race,driver_gender_driver_race_state,F_Asian_IL,M_Black_RI,[Asian],[Black],NaN,NaN,rank_trend
950457,NaN,NaN,pairwise,hit_rate,driver_gender,driver_gender_driver_race_state,F_Asian_WA,F_Other_WA,[F],[F],NaN,NaN,rank_trend
1454891,NaN,NaN,pairwise,year,num_stops,driver_gender_driver_race_state,F_Black_WI,M_Black_RI,-26.7857,245.5,0.037388,0.627203,lin_reg
1297026,NaN,NaN,pairwise,year,num_stops,driver_race_state,Hispanic_NC,Other_MA,829.488,319.906,0.267606,0.324592,lin_reg
218749,NaN,NaN,pairwise,hit_rate,driver_race,state,MA,WI,"[Asian, Other, Hispanic, Black, White]","[Hispanic, Asian, Black, Other, White]",0.075500,0.201900,rank_trend


In [23]:
pd.unique(labeled_df.result_df['comparison_type'])

array(['aggregate-subgroup', 'pairwise'], dtype=object)

The object also stores the trend objects that have been applied, they can be used for mapping to get the distance functions that are appropriate for each trend

In [24]:
labeled_df.trend_list

In [25]:
labeled_df.result_df.head()

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type
0,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,AZ,NaN,-0.247018,NaN,0.247018,NaN,pearson_corr
1,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,CO,NaN,-0.414566,NaN,0.414566,NaN,pearson_corr
2,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,CT,NaN,0.118238,NaN,0.118238,NaN,pearson_corr
3,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,IL,NaN,-0.199765,NaN,0.199765,NaN,pearson_corr
4,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,MA,NaN,-0.603026,NaN,0.603026,NaN,pearson_corr


In [26]:
# a = ['a','b','c','d']
# b = ['a','b','d','e']
a = ['F', 'M']
b = ['F', 'M']
for i, d in enumerate(set(a+b)):
    print (i,':',d)

0 : M
1 : F


In [27]:
# labeled_df.result_df['distance'] = labeled_df.result_df.apply(dist_helper,axis=1)
labeled_df.add_distance(row_wise=True) #('subgroup_trend','subgroup_trend2')

labeled_df.result_df.sample(10)

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance
228000,NaN,NaN,pairwise,search_conducted_rate,state,driver_gender_state,M_IL,M_WA,[IL],[WA],NaN,NaN,rank_trend,NaN
564709,NaN,NaN,pairwise,search_conducted_rate,driver_gender,driver_gender_driver_race_state,F_Other_CO,M_Hispanic_MA,[F],[M],NaN,NaN,rank_trend,NaN
214729,NaN,NaN,pairwise,year,search_conducted_rate,search_conducted_rate_num_stops_dpgmm,4,5,0.0713551,-0.0985676,0.071355,0.098568,pearson_corr,1.000000
918267,NaN,NaN,pairwise,hit_rate,driver_gender,driver_gender_driver_race_state,F_Black_AZ,M_White_AZ,[F],[M],NaN,NaN,rank_trend,NaN
1376293,NaN,NaN,pairwise,num_stops,contraband_found_rate,driver_race_state,Asian_WA,Black_CO,0.00123445,0.000802667,4.806583,2.865116,lin_reg,0.000275
60292,NaN,NaN,pairwise,year,search_conducted_rate,driver_gender_driver_race_state,M_Asian_VT,M_Black_WI,-0.524556,0.0183395,0.524556,0.018340,pearson_corr,1.000000
1147249,NaN,NaN,pairwise,num_stops,driver_gender,driver_gender_driver_race_state,F_Asian_WI,M_Other_IL,[F],[M],NaN,NaN,rank_trend,NaN
1113125,NaN,NaN,pairwise,num_stops,driver_gender,driver_gender_driver_race_state,F_Asian_NC,F_Hispanic_MA,[F],[F],NaN,NaN,rank_trend,NaN
1461292,NaN,NaN,pairwise,year,num_stops,driver_gender_driver_race_state,F_Other_WA,M_Black_SC,-101.19,-1213.13,0.174514,0.163904,lin_reg,0.005766
499424,NaN,NaN,pairwise,search_conducted_rate,state,driver_gender_driver_race_state,F_Other_CT,M_Black_WI,[CT],[WI],NaN,NaN,rank_trend,NaN


Each trend object has a trend_precompute dictionary as a property that stores the intermediate values (tables of the weighted rates for ranks and correlation matrices for pearson correlation, TODO: what do we need for linreg). These can be used in vizualization.

In [28]:
labeled_df.trend_list[0].trend_precompute

{'pearson_corr_agg_trend':                            year  search_conducted_rate  contraband_found_rate  \
 year                   1.000000              -0.039030               0.041129   
 search_conducted_rate -0.039030               1.000000               0.807915   
 contraband_found_rate  0.041129               0.807915               1.000000   
 hit_rate               0.316196              -0.094273               0.275397   
 num_stops             -0.043965              -0.065594              -0.073456   
 
                        hit_rate  num_stops  
 year                   0.316196  -0.043965  
 search_conducted_rate -0.094273  -0.065594  
 contraband_found_rate  0.275397  -0.073456  
 hit_rate               1.000000  -0.042102  
 num_stops             -0.042102   1.000000  ,
 'pearson_corr_subgroup_trend':                                               year  search_conducted_rate  \
 num_stopsquantiles                                                          
 high           

# Saving with trends

In [29]:
labeled_df.save_all('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI_all')

True

In [30]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI_all
ls

df.csv
meta.csv
result_df.csv
trends.json


In [31]:
labeled_df = wg.LabeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI_all')

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (0,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/pandas/core/frame.py:3489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [32]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [33]:
labeled_df.trend_list[0].trend_precompute

{'pearson_corr_agg_trend':        year  search_conducted_rate  contraband_found_rate  hit_rate  num_stops
 0  1.000000              -0.039030               0.041129  0.316196  -0.043965
 1 -0.039030               1.000000               0.807915 -0.094273  -0.065594
 2  0.041129               0.807915               1.000000  0.275397  -0.073456
 3  0.316196              -0.094273               0.275397  1.000000  -0.042102
 4 -0.043965              -0.065594              -0.073456 -0.042102   1.000000,
 'pearson_corr_subgroup_trend':         year  search_conducted_rate  contraband_found_rate  hit_rate  \
 0   1.000000               0.011983               0.066080  0.258733   
 1   0.011983               1.000000               0.729171 -0.233576   
 2   0.066080               0.729171               1.000000  0.255356   
 3   0.258733              -0.233576               0.255356  1.000000   
 4  -0.032069               0.047615               0.107314  0.002882   
 5   1.000000           

# Filtering

In [34]:
help(labeled_df.get_trend_rows)

Help on method get_trend_rows in module wiggum.ranking_processing:

get_trend_rows(feat1=None, feat2=None, group_feat=None, subgroup=None, subgroup2=None, trend_type=None, comparison_type=None, inplace=False) method of wiggum.labeled_dataframe.LabeledDataFrame instance
    return a row of result_df based on the specified values. returned rows
    meet provided criteria for all columns (and operator) and any one of the listed
    values for each column (or operator)
    
    Parameters
    -----------
    feat1 : str, list, or  {None}
        trend variable name or None to include all
    feat2 : str, list, or  {None}
        trend variable name or None to include all
    group_feat : str, list, or  {None}
        groupoby variable name or None to include all
    subgroup : str, list, or  {None}
        value of groupby_feat or  None to include all
    subgroup2 : str, list or {None}
        value of groupby_feat or  None to include all
    trend_type: str, list or {None}
        name o

So, we can use that function to filter and look at subsets of the trends based on the features, groupby, or subgroups

In [35]:
labeled_df.get_trend_rows(feat1='year',subgroup=['Black','Hispanic'])

72  total rows meet the criteria


,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance
5753,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,Black,NaN,-2262.72,NaN,0.169689,NaN,lin_reg,0.000127
5754,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,Hispanic,NaN,422.667,NaN,0.014732,NaN,lin_reg,0.998086
6016,1.96425,0.004473,aggregate-subgroup,year,search_conducted_rate,driver_race,Black,NaN,1.24021,NaN,0.001744,NaN,lin_reg,0.132212
6017,1.96425,0.004473,aggregate-subgroup,year,search_conducted_rate,driver_race,Hispanic,NaN,5.34096,NaN,0.017419,NaN,lin_reg,0.181954
613682,NaN,NaN,pairwise,year,num_stops,driver_race,Black,Hispanic,-2262.72,422.667,0.169689,0.014732,lin_reg,0.998212
613683,NaN,NaN,pairwise,year,num_stops,driver_race,Black,Other,-2262.72,91.5127,0.169689,0.075180,lin_reg,0.992762
613684,NaN,NaN,pairwise,year,num_stops,driver_race,Black,White,-2262.72,-5591.73,0.169689,0.097383,lin_reg,0.000168
613685,NaN,NaN,pairwise,year,num_stops,driver_race,Black,Hispanic,-2262.72,422.667,0.169689,0.014732,lin_reg,0.998212
613686,NaN,NaN,pairwise,year,num_stops,driver_race,Black,Other,-2262.72,91.5127,0.169689,0.075180,lin_reg,0.992762
613687,NaN,NaN,pairwise,year,num_stops,driver_race,Black,White,-2262.72,-5591.73,0.169689,0.097383,lin_reg,0.000168


In [36]:
labeled_df.get_trend_rows(group_feat = 'driver_race',trend_type ='lin_reg' )

645  total rows meet the criteria


,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance
5752,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,Asian,NaN,259.628,NaN,0.104919,NaN,lin_reg,0.997140
5753,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,Black,NaN,-2262.72,NaN,0.169689,NaN,lin_reg,0.000127
5754,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,Hispanic,NaN,422.667,NaN,0.014732,NaN,lin_reg,0.998086
5755,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,Other,NaN,91.5127,NaN,0.075180,NaN,lin_reg,0.992636
5756,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,White,NaN,-5591.73,NaN,0.097383,NaN,lin_reg,0.000294
6015,1.96425,0.004473,aggregate-subgroup,year,search_conducted_rate,driver_race,Asian,NaN,1.24187,NaN,0.003783,NaN,lin_reg,0.131795
6016,1.96425,0.004473,aggregate-subgroup,year,search_conducted_rate,driver_race,Black,NaN,1.24021,NaN,0.001744,NaN,lin_reg,0.132212
6017,1.96425,0.004473,aggregate-subgroup,year,search_conducted_rate,driver_race,Hispanic,NaN,5.34096,NaN,0.017419,NaN,lin_reg,0.181954
6018,1.96425,0.004473,aggregate-subgroup,year,search_conducted_rate,driver_race,Other,NaN,-7.23536,NaN,0.032087,NaN,lin_reg,0.612781
6019,1.96425,0.004473,aggregate-subgroup,year,search_conducted_rate,driver_race,White,NaN,1.9543,NaN,0.003678,NaN,lin_reg,0.001309


In [37]:
labeled_df.result_df.columns

Index(['agg_trend', 'agg_trend_strength', 'comparison_type', 'feat1', 'feat2',
       'group_feat', 'subgroup', 'subgroup2', 'subgroup_trend',
       'subgroup_trend2', 'subgroup_trend_strength',
       'subgroup_trend_strength2', 'trend_type', 'distance'],
      dtype='object')

We can also filter based on SP detections with `

In [38]:
labeled_df.get_SP_rows(thresh=.2)

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2
5737,-1560.86,0.043965,aggregate-subgroup,year,num_stops,state,AZ,NaN,8474.29,NaN,0.301916,NaN,lin_reg,0.999517,True
5741,-1560.86,0.043965,aggregate-subgroup,year,num_stops,state,MA,NaN,2227.27,NaN,0.115809,NaN,lin_reg,0.999306,True
5743,-1560.86,0.043965,aggregate-subgroup,year,num_stops,state,NC,NaN,2120.79,NaN,0.117245,NaN,lin_reg,0.999292,True
5744,-1560.86,0.043965,aggregate-subgroup,year,num_stops,state,RI,NaN,135.696,NaN,0.058311,NaN,lin_reg,0.994901,True
5747,-1560.86,0.043965,aggregate-subgroup,year,num_stops,state,VT,NaN,271.488,NaN,0.044411,NaN,lin_reg,0.997247,True
5752,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,Asian,NaN,259.628,NaN,0.104919,NaN,lin_reg,0.997140,True
5754,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,Hispanic,NaN,422.667,NaN,0.014732,NaN,lin_reg,0.998086,True
5755,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,Other,NaN,91.5127,NaN,0.075180,NaN,lin_reg,0.992636,True
5757,-1560.86,0.043965,aggregate-subgroup,year,num_stops,decriminalization,0,NaN,637.751,NaN,0.025548,NaN,lin_reg,0.998594,True
5759,-1560.86,0.043965,aggregate-subgroup,year,num_stops,medical,0,NaN,1794.75,NaN,0.044001,NaN,lin_reg,0.999237,True


## Detection

Detection via `get_SP_rows` happens in two steps:
1. label the rows
2. filter by that column to return

Labeling the rows can happen in a number of ways too, the detection accepts a number of forms of input, custom detections can be built in many ways

In [39]:
help(labeled_df.label_SP_rows)

Help on method label_SP_rows in module wiggum.ranking_processing:

label_SP_rows(filter_thresh=None) method of wiggum.labeled_dataframe.LabeledDataFrame instance
    update the result_df with an additional colulmn indicateing rows with SP
    (or SP-like) as defined by sp_type
    
    Parameters
    -----------
    
    self : LabeledDataFrame
        must have values in result_df
    filter_thresh : dict or string
        dictionary of column label, threshold pairs or string name of a
        prespecified dictionary if dict, must include 'name' field (which
        will be used as the column name for storing the detections)



when filter_thresh is a dictionary, the filtering happens by taking the intersection of each row by the treshold prvided.  Some defaults are also built in accessible by string.

In [40]:
wg.trend_quality_sp

{'distance': 0.2,
 'agg_trend_strength': 0.15,
 'subgroup_trend_strength': 0.15,
 'name': 'default_qual_sp'}

Which can be applied with:

In [41]:
labeled_df.get_SP_rows('default_qual_sp')

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp
532,0.316196,0.316196,aggregate-subgroup,year,hit_rate,state,SC,NaN,-0.255096,NaN,0.255096,NaN,pearson_corr,1.0000,True,True
535,0.316196,0.316196,aggregate-subgroup,year,hit_rate,state,WA,NaN,-0.68403,NaN,0.684030,NaN,pearson_corr,1.0000,True,True
564,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_gender_state,F_CT,NaN,-0.262423,NaN,0.262423,NaN,pearson_corr,1.0000,True,True
570,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_gender_state,F_SC,NaN,-0.253289,NaN,0.253289,NaN,pearson_corr,1.0000,True,True
573,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_gender_state,F_WA,NaN,-0.646752,NaN,0.646752,NaN,pearson_corr,1.0000,True,True
575,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_gender_state,M_AZ,NaN,-0.15601,NaN,0.156010,NaN,pearson_corr,1.0000,True,True
583,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_gender_state,M_SC,NaN,-0.264436,NaN,0.264436,NaN,pearson_corr,1.0000,True,True
586,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_gender_state,M_WA,NaN,-0.768069,NaN,0.768069,NaN,pearson_corr,1.0000,True,True
598,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_race_state,Asian_WA,NaN,-0.723905,NaN,0.723905,NaN,pearson_corr,1.0000,True,True
599,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_race_state,Asian_WI,NaN,-0.317996,NaN,0.317996,NaN,pearson_corr,1.0000,True,True


In [42]:
wg.DEFAULT_SP_DEF

{'distance': 0.0, 'name': 'SP', 'comparison_type': 'aggregate'}

Which can be applied with:

In [43]:
labeled_df.get_SP_rows('SP')

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp,SP


We can also define our own detection filters, using any available column

In [44]:
lin_only_qual = {'name':'lin_only_qual_sp','distance':.2, 'agg_trend_strength':.05,
                'subgroup_trend_strength':.15,'trend_type':'lin_reg'}
labeled_df.get_SP_rows(lin_only_qual,replace=True) 

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp,SP,lin_only_qual_sp


# Ranking

In [45]:
labeled_df.rank_occurences_by_view(ascending=False).head(20)

Index(['agg_trend', 'agg_trend_strength', 'comparison_type', 'feat1', 'feat2',
       'group_feat', 'subgroup', 'subgroup2', 'subgroup_trend',
       'subgroup_trend2', 'subgroup_trend_strength',
       'subgroup_trend_strength2', 'trend_type', 'distance', 'SP_thresh0.2',
       'default_qual_sp', 'SP', 'lin_only_qual_sp', 'mean_view_distance'],
      dtype='object')


,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp,SP,lin_only_qual_sp,mean_view_distance
692255,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_gender_state,F_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7000,True,False,False,False,0.522039
692268,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_gender_state,M_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7000,True,False,False,False,0.522039
692280,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_race_state,Asian_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7000,True,False,False,False,0.522039
692293,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_race_state,Black_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7000,True,False,False,False,0.522039
692306,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_race_state,Hispanic_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7000,True,False,False,False,0.522039
692319,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_race_state,Other_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7000,True,False,False,False,0.522039
692332,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_race_state,White_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7000,True,False,False,False,0.522039
692344,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_gender_driver_race_state,F_Asian_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7000,True,False,False,False,0.522039
692357,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_gender_driver_race_state,F_Black_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7000,True,False,False,False,0.522039
692370,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_gender_driver_race_state,F_Hispanic_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7000,True,False,False,False,0.522039


In [46]:
labeled_df.add_view_score('SP_thresh0.2',agg_type='sum',colored=False).head(10)

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp,SP,lin_only_qual_sp,mean_view_distance,sum_view_SP_thresh0.2
0,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_gender_state,F_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7,True,False,False,False,0.522039,572.0
1,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_gender_state,M_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7,True,False,False,False,0.522039,572.0
2,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_race_state,Asian_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7,True,False,False,False,0.522039,572.0
3,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_race_state,Black_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7,True,False,False,False,0.522039,572.0
4,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_race_state,Hispanic_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7,True,False,False,False,0.522039,572.0
5,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_race_state,Other_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7,True,False,False,False,0.522039,572.0
6,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_race_state,White_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7,True,False,False,False,0.522039,572.0
7,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_gender_driver_race_state,F_Asian_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7,True,False,False,False,0.522039,572.0
8,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_gender_driver_race_state,F_Black_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7,True,False,False,False,0.522039,572.0
9,['NC' 'CO' 'MD' 'WA' 'CT' 'SC' 'TX' 'WI' 'IL' ...,0.0201,aggregate-subgroup,contraband_found_rate,state,driver_gender_driver_race_state,F_Hispanic_TX,NaN,['TX'],nan,NaN,NaN,rank_trend,0.7,True,False,False,False,0.522039,572.0


In [47]:
labeled_df.rank_occurences_by_view('sum_view_SP_thresh0.2','SP_thresh0.2').head()

Index(['agg_trend', 'agg_trend_strength', 'comparison_type', 'feat1', 'feat2',
       'group_feat', 'subgroup', 'subgroup2', 'subgroup_trend',
       'subgroup_trend2', 'subgroup_trend_strength',
       'subgroup_trend_strength2', 'trend_type', 'distance', 'SP_thresh0.2',
       'default_qual_sp', 'SP', 'lin_only_qual_sp', 'mean_view_distance',
       'sum_view_SP_thresh0.2'],
      dtype='object')


,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp,SP,lin_only_qual_sp,mean_view_distance,sum_view_SP_thresh0.2
483099,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,CT,NaN,0.118238,NaN,0.118238,NaN,pearson_corr,1.0,True,False,False,False,0.387097,31664.0
483100,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,SC,NaN,0.000440801,NaN,0.000441,NaN,pearson_corr,1.0,True,False,False,False,0.387097,31664.0
483101,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,WI,NaN,0.0485728,NaN,0.048573,NaN,pearson_corr,1.0,True,False,False,False,0.387097,31664.0
483102,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,driver_gender,F,NaN,0.0304108,NaN,0.030411,NaN,pearson_corr,1.0,True,False,False,False,0.387097,31664.0
483103,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,driver_race,Other,NaN,0.134696,NaN,0.134696,NaN,pearson_corr,1.0,True,False,False,False,0.387097,31664.0
